# QF605 Fixed Income Securities
## Project Part IV
## Decompounded Options
YU Lingfeng

In [57]:
# importing labraries
import numpy as np
import datetime as dt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import seaborn as sns
import math
from scipy.optimize import brentq, fsolve, least_squares
from scipy.stats import norm
from scipy.integrate import quad
import warnings

# Ignore all warnings
warnings.filterwarnings("ignore")

## <a id = "top">Table of Content</a>
### [Q1. Decompounded Option 1 payoff:](#p1)   

* $T=5y, p=4, q=2$

### $$CMS 10y^{1/p}-0.04^{1/q}$$

### [Q2. Decompounded Option 2 payoff:](#p2)   

* $T=5y, p=4, q=2$

### $$(CMS 10y^{1/p}-0.04^{1/q})^+$$


In [58]:
def Black76Call(F, K, sigma, T):
    d1 = (np.log(F/K) + 1/2 *(sigma**2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    return F*norm.cdf(d1) - K*norm.cdf(d2)

def Black76Put(F, K, sigma, T):
    d1 = (np .log(F/K) + 1/2 * sigma**2*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    return K*norm.cdf(-d2) - F*norm.cdf(-d1)

def SABR(F, K, T, alpha, beta, rho, nu):
    X = K
    # if K is at-the-money-forward
    if abs(F - K) < 1e-12:
        numer1 = (((1 - beta)**2)/24)*alpha*alpha/(F**(2 - 2*beta))
        numer2 = 0.25*rho*beta*nu*alpha/(F**(1 - beta))
        numer3 = ((2 - 3*rho*rho)/24)*nu*nu
        VolAtm = alpha*(1 + (numer1 + numer2 + numer3)*T)/(F**(1-beta))
        sabrsigma = VolAtm
    else:
        z = (nu/alpha)*((F*X)**(0.5*(1-beta)))*np.log(F/X)
        zhi = np.log((((1 - 2*rho*z + z*z)**0.5) + z - rho)/(1 - rho))
        numer1 = (((1 - beta)**2)/24)*((alpha*alpha)/((F*X)**(1 - beta)))
        numer2 = 0.25*rho*beta*nu*alpha/((F*X)**((1 - beta)/2))
        numer3 = ((2 - 3*rho*rho)/24)*nu*nu
        numer = alpha*(1 + (numer1 + numer2 + numer3)*T)*z
        denom1 = ((1 - beta)**2/24)*(np.log(F/X))**2
        denom2 = (((1 - beta)**4)/1920)*((np.log(F/X))**4)
        denom = ((F*X)**((1 - beta)/2))*(1 + denom1 + denom2)*zhi
        sabrsigma = numer/denom

    return sabrsigma


In [59]:
### By numeric method
### Lecture 5, Page 5/22

#IRR function 
def IRR_0(x,N,m):
    IRR=np.zeros(N*m)
    IRRS=0
    for i in range(N*m):
        IRR[i]= 1/m / (1+x/m)**i
    IRRS=np.sum(IRR[:])
    return IRRS
# first derivative of IRR
def IRR_1(x,N,m):
    dx = 0.00001 * x
    IRRplus= IRR_0(x+dx,N,m)
    IRRminus = IRR_0(x-dx,N,m)
    IRRf = (IRRplus - IRRminus) / (2*dx)
    return IRRf

# second derivative of IRR
def IRR_2(x,N,m):
    dx = 0.00001 * x
    IRRplus= IRR_0(x+dx,N,m)
    IRRx = IRR_0(x,N,m)
    IRRminus = IRR_0(x-dx,N,m)
    IRRff = (IRRplus - 2*IRRx + IRRminus) / (dx**2)
    return IRRff

In [60]:
### by close form formula
def IRR_0(K, m, N):
    # implementation of IRR(K) function
    value = 1/K * ( 1.0 - 1/(1 + K/m)**(N*m) )
    return value

def IRR_1(K, m, N):
    # implementation of IRR'(K) function (1st derivative)
    firstDerivative =\
        -1/K*IRR_0(K, m, N) +\
        1/(K*m)*N*m/(1+K/m)**(N*m+1)
    return firstDerivative

def IRR_2(K, m, N):
    # implementation of IRR''(K) function (2nd derivative)
    secondDerivative =\
        -2/K*IRR_1(K, m, N) -\
        1/(K*m*m)*(N*m)*(N*m+1)/(1+K/m)**(N*m+2)
    return secondDerivative

In [61]:
### Lecture 5, Page 18/22
# p = 4 and q = 2
# g(K) = x^(1/p) - 0.04^(1/q)
# payoff function and derivatives 
def g_0(x, p=4, q=2):
    return x**(1/p) - 0.04**(1/q)

def g_1(x, p=4, q=2):
    return (1/p) * x ** (1/p - 1)

def g_2(x, p=4, q=2):
    return (1/p * (1/p - 1)) * x ** (1/p - 2)

def g_all(x, p=4, q=2):
    g0 = x**(1/p) - 0.04**(1/q)
    g1 = (1/p) * x ** (1/p - 1)
    g2 = (1/p * (1/p - 1)) * x ** (1/p - 2)
    return [g0, g1, g2]

In [62]:
def h_0(K, m, N):
    # implementation of h(K)
    value = g_0(K) / IRR_0(K, m, N)
    return value

def h_1(K, m, N):
    # implementation of h'(K) (1st derivative)
    firstDerivative =\
        (IRR_0(K, m, N)*g_1(K) - g_0(K)*IRR_1(K, m, N)) /\
        IRR_0(K, m, N)**2
    return firstDerivative

def h_2(K, m, N):
    # implementation of h''(K) (2nd derivative)
    g = g_all(K)
    secondDerivative =\
        ((IRR_0(K, m, N)*g_2(K) - IRR_2(K, m, N)*g_0(K) -\
          2.0*IRR_1(K, m, N)*g_1(K))/IRR_0(K, m, N)**2 +\
         2.0*IRR_1(K, m, N)**2*g_0(K)/IRR_0(K, m, N)**3)
    return secondDerivative

In [63]:
# ### Lecture 5, Page 17/22

# receiver payoff convexity part for strike range [0, F]
def I_put(x,m,N,F,sigma,T):
    h = h_2(x, m, N)
    Vrec = Black76Put(F, x, sigma, T)
    return h*Vrec
# payer swaptions convexity part for strike range [F, np.inf]
def I_call(x,m,N,F,sigma,T):
    h = h_2(x, m, N)
    Vpay = Black76Call(F, x, sigma, T)
    return h*Vpay

In [64]:
CMS10y =\
    pd.read_csv("###P3 CMS 10yX5y semi.csv", 
                index_col=[0])
CMS10y

,Tenor,Forward Swap,OIS DF,alpha,beta,rho,nu,CMS
Semi Annual,,,,,,,,
0.5,10,0.037845,0.998752,0.170308,0.9,-0.242848,0.813334,0.037820
1.0,10,0.038428,0.997009,0.171053,0.9,-0.264432,0.778196,0.038368
1.5,10,0.039020,0.995270,0.171797,0.9,-0.286017,0.743059,0.038952
2.0,10,0.039634,0.993531,0.172542,0.9,-0.307602,0.707921,0.039637
2.5,10,0.040200,0.991773,0.173287,0.9,-0.329186,0.672783,0.040371
3.0,10,0.040788,0.990015,0.174031,0.9,-0.350771,0.637645,0.041172
3.5,10,0.041412,0.988066,0.174776,0.9,-0.372355,0.602508,0.041959
4.0,10,0.042062,0.986117,0.175521,0.9,-0.393940,0.567370,0.042665
4.5,10,0.042831,0.984150,0.176265,0.9,-0.415525,0.532232,0.043363


## <a id = "p1"> Q1. </a> $T=5y, p=4, q=2$ [back to table of contents](#top)

### $$CMS 10y^{1/p}-0.04^{1/q}$$

In [65]:
p = 4; q = 2; T = 5
F = CMS10y.loc[T, 'Forward Swap']
DF = CMS10y.loc[T, 'OIS DF']
alpha = CMS10y.loc[T, 'alpha']
beta =CMS10y.loc[T, 'beta']
rho = CMS10y.loc[T, 'rho']
nu = CMS10y.loc[T, 'nu']

sigma = SABR(F, F, T, alpha, beta, rho, nu)
F, DF, alpha, beta, rho, nu, sigma

(0.0436336455274834,
 0.9821841197332212,
 0.1770099910248877,
 0.9,
 -0.4371092158056472,
 0.497094422726149,
 0.24559579386018027)

In [66]:
m = 2; N = 10
term1 =\
    quad(lambda x: I_put(x, m, N, F,
                         SABR(F, x, T, alpha, beta, rho, nu), T), 0, F)
term2 =\
    quad(lambda x: I_call(x, m, N, F,
                          SABR(F, x, T, alpha, beta, rho, nu), T), F, np.inf)
PVoption1 = ((g_0(F)*DF + term1[0] + term1[0]))
PVoption1

0.2467176784952853

## <a id = "p2"> Q2. </a> $T=5y, p=4, q=2$ [back to table of contents](#top)

### $$(CMS 10y^{1/p}-0.04^{1/q})^+$$

In [67]:
L = 0.04**(p/q)
L

0.0016

In [68]:
term3 =\
    quad(lambda x: I_put(x, m, N, F,
                         SABR(F, x, T, alpha, beta, rho, nu), T), L, F)
term4 =\
    quad(lambda x: I_call(x, m, N, F,
                          SABR(F, x, T, alpha, beta, rho, nu), T), F, np.inf)
PVoption2 = ((g_0(F)*DF + term3[0] + term4[0]))
PVoption2

0.25132624379160867